In [1]:
import heapq
from itertools import combinations

In [ ]:
with open("input-example.txt") as f:
    lines = f.readlines()
lines = [l.strip() for l in lines]
lines[:5]

In [ ]:
start_node = (0, 0)
target_node = (0, 0)
for i in range(0, len(lines)):
    for j in range(0, len(lines[0])):
        if lines[i][j] == "E":
            target_node = (i, j)
        if lines[i][j] == "S":
            start_node = (i, j)
target_node

In [4]:
dirs = [(-1, 0), (0, 1), (1, 0), (0, -1)]

# Get dirs to turn to (180 makes no sense)
turn_dict = {
    (-1, 0): [(0, -1), (0, 1)],
    (0, 1): [(-1, 0), (1, 0)],
    (1, 0): [(0, 1), (0, -1)],
    (0, -1): [(1, 0), (-1, 0)],
}


def walk(pos, dir):
    return (pos[0] + dir[0], pos[1] + dir[1])


def in_bounds(pos, field):
    return (
        pos[0] >= 0 and pos[0] < len(field) and pos[1] >= 0 and pos[1] < len(field[0])
    )


def print_field(
    field,
):
    i = 0
    for l in field:
        j = 0
        for p in l:
            print(p, end="")
            j += 1
        print()
        i += 1


# Expand all previous nodes until everything is expanded and return all distinct found nodes
def get_path(target, distances, prev_nodes):
    expanding_nodes = [(k) for k, v in distances.items() if k == target]
    prevs = []
    while expanding_nodes:
        n = expanding_nodes.pop()
        prev = prev_nodes[n]
        prevs += [n]
        if not prev:
            continue
        expanding_nodes += [prev_nodes[n]]
    return list(reversed([n for n in prevs]))


def print_maze(field, path_nodes):
    i = 0
    for l in field:
        j = 0
        for p in l:
            if (i, j) in path_nodes and p not in ["S", "E"]:
                print("O", end="")
            else:
                print(p, end="")
            j += 1
        print()
        i += 1


def get_manhatten_dist(a, b):
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

In [5]:
# Cheats are a dict between 2 points on the path with a distance of at most max_dist, as we can remove everything in between if needed
# Pre-compute a dict that holds for each field all reachable fields through cheats with the distance travelled
# By looking at each combination of fields only once, cheats are unique between points with the smallest possible distance
def get_cheats(max_dist, field):
    way_nodes = []
    for i in range(0, len(field)):
        for j in range(0, len(field[0])):
            if field[i][j] != "#":
                way_nodes.append((i, j))
    # print(way_nodes)
    way_node_combinations = combinations(way_nodes, 2)
    cheatable_combinations = [
        c
        for c in way_node_combinations
        if get_manhatten_dist(c[0], c[1]) <= max_dist
        and get_manhatten_dist(c[0], c[1]) > 1
    ]
    # print(len(cheatable_combinations), cheatable_combinations)
    cheat_dict = {}
    for cmb in cheatable_combinations:
        dst = get_manhatten_dist(cmb[0], cmb[1])
        if cmb[0] not in cheat_dict:
            cheat_dict[cmb[0]] = []
        if cmb[1] not in cheat_dict:
            cheat_dict[cmb[1]] = []
        cheat_dict[cmb[0]].append((cmb[1], dst))
        cheat_dict[cmb[1]].append((cmb[0], dst))
    return cheat_dict


# get_cheats(2,lines)#[(3,1)]

In [6]:
def dijkstra(start, target, field):
    priority_queue = []
    heapq.heappush(priority_queue, (0, start))
    prev_nodes = {}
    distances = {}

    distances[start] = 0
    prev_nodes[start] = None
    visited_nodes = set()

    while priority_queue:
        current_distance, current_node = heapq.heappop(priority_queue)
        visited_nodes.add(current_node)
        if current_node == target:
            break

        if current_distance > distances.get(current_node, float("inf")):
            continue

        # Get neighbours
        neighbours = []
        # print(current_node)
        for dir in dirs:
            next_node = walk(current_node, dir)
            if (
                in_bounds(next_node, field)
                and field[next_node[0]][next_node[1]] != "#"
                and next_node not in visited_nodes
            ):
                neighbours.append((1, next_node))

        for weight, neighbor in neighbours:
            distance = current_distance + weight
            if distance < distances.get(neighbor, float("inf")):
                distances[neighbor] = distance
                prev_nodes[neighbor] = current_node
                heapq.heappush(priority_queue, (distance, neighbor))

    return distances, prev_nodes

In [ ]:
# Do Dijkstra once as we already have it implemented to get path and baseline dist without cheats
field = []
for i in range(0, len(lines)):
    l = []
    for j in range(0, len(lines[0])):
        l.append(lines[i][j])
    field.append(l)

distances, prev_nodes = dijkstra(start_node, target_node, field)
baseline_dist = distances[target_node]
baseline_dist

In [8]:
# First idea was to remove every single boundary once as thats a cheat of len 2 and recompute dijkstra
# This is very brute force and not very smart and is already very slow on p1
# It being very brute force made me refactor everything

# shortcut_cnt = 0
# for k in range(1, len(lines)-1):
#     for l in range(1, len(lines[0])-1):

#         field = []
#         for i in range(0, len(lines)):
#             ls = []
#             for j in range(0, len(lines[0])):

#                 if i == k and j==l and lines[i][j] == '#':
#                     ls.append('.')
#                 else:
#                     ls.append(lines[i][j])
#             field.append(ls)

#         distances, prev_nodes = dijkstra(start_node, target_node, field)
#         dist = distances[target_node]
#         if baseline_dist - dist > 99:
#             # print(distances[target_node], baseline_dist - dist)
#             shortcut_cnt += 1
# shortcut_cnt

In [9]:
# print_maze(lines, get_path(target_node, distances, prev_nodes))

In [10]:
# The new idea is that since it is only one path, cheats can't open up other paths
# We can pre-compute all cheats reachable from every point by checking all combinations of this point with other path fields with manhatten distance smaller the allowed cheat
# And then we can walk the path and check on each field if going through one of the cheats reduces the travelled distance enough to count for the challenge
# Using the path and precomputed distances from the single dijkstra run as well as the cheat dists computed during creation of the cheat dict, this is fairly fast

In [ ]:
normal_path = get_path(target_node, distances, prev_nodes)
baseline_distances = {k: (baseline_dist - v) for k, v in distances.items()}
possible_cheats = get_cheats(2, lines)

walked_dist = 0
good_cheat_count = 0
for np in normal_path:
    cheats_from_np = possible_cheats[np]
    # print(cheats_from_np)
    for ch in cheats_from_np:
        dist_with_cheat = walked_dist + baseline_distances[ch[0]] + ch[1]
        # print(np, ch, dist_with_cheat)
        if (baseline_dist - dist_with_cheat) > 99:
            good_cheat_count += 1
    walked_dist += 1
good_cheat_count

In [12]:
# part 2

In [13]:
# From refactored part 1, part 2 is basically free

In [ ]:
normal_path = get_path(target_node, distances, prev_nodes)
baseline_distances = {k: (baseline_dist - v) for k, v in distances.items()}
possible_cheats = get_cheats(20, lines)

walked_dist = 0
good_cheat_count = 0
for np in normal_path:
    cheats_from_np = possible_cheats[np]
    # print(cheats_from_np)
    for ch in cheats_from_np:
        dist_with_cheat = walked_dist + baseline_distances[ch[0]] + ch[1]
        # print(np, ch, dist_with_cheat)
        if (baseline_dist - dist_with_cheat) > 99:
            good_cheat_count += 1
    walked_dist += 1
good_cheat_count